<a href="https://colab.research.google.com/github/Mechanics-Mechatronics-and-Robotics/ML-2025a/blob/main/Week_13/PINN_Couette.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization

In [1]:
# Install dependencies for Colab
!pip install pytorch-lightning clearml

import torch
import torch.nn as nn
import pytorch_lightning as pl
from clearml import Task

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 832.4/832.4 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 52.2 MB/s eta 0:00:00


In [2]:
#Enter your code here to implement Step 2 of the logging instruction as it is shown below
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=ZP02U03C6V5ER4K9VWRNZT7EWA5ZTV
%env CLEARML_API_SECRET_KEY=BtA5GXZufr6QGpaqhX1GSKPTvaCt56OLqaNqUGLNoxx2Ye8Ctwbui0Ln5OXVnzUgH4I

env: CLEARML_WEB_HOST=https://app.clear.ml/
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=ZP02U03C6V5ER4K9VWRNZT7EWA5ZTV
env: CLEARML_API_SECRET_KEY=BtA5GXZufr6QGpaqhX1GSKPTvaCt56OLqaNqUGLNoxx2Ye8Ctwbui0Ln5OXVnzUgH4I


# Model

In [3]:


# Initialize ClearML
task = Task.init(project_name='PINN_Project', task_name='Parallel Plates Flow PINN')

# PINN Model (a simple fully-connected NN)
class PINN(pl.LightningModule):
    def __init__(self, layers, mu, dpdx):
        super().__init__()
        self.mu = mu           # viscosity
        self.dpdx = dpdx       # pressure gradient (can be learnable)
        layer_list = []
        for i in range(len(layers)-1):
            layer_list.append(nn.Linear(layers[i], layers[i+1]))
        self.layers = nn.ModuleList(layer_list)
        self.activation = nn.Tanh()

    def forward(self, y):
        out = y
        for i in range(len(self.layers)-1):
            out = self.activation(self.layers[i](out))
        out = self.layers[-1](out)
        return out

    def pinn_residual(self, y):
        """Calculate PDE residual: mu * u'' - dp/dx"""
        y = y.requires_grad_(True)
        u = self.forward(y)
        u_y = torch.autograd.grad(u, y, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), create_graph=True)[0]
        residual = self.mu * u_yy - self.dpdx
        return residual

    def training_step(self, batch, batch_idx):
        y_bc, u_bc = batch['y_bc'], batch['u_bc']
        y_colloc = batch['y_colloc']

        # Boundary condition loss (no-slip)
        u_pred_bc = self.forward(y_bc)
        bc_loss = nn.functional.mse_loss(u_pred_bc, u_bc)
        self.log('bc_loss', bc_loss)

        # PDE residual loss
        residual = self.pinn_residual(y_colloc)
        pde_loss = torch.mean(residual**2)
        self.log('pde_loss', pde_loss)

        loss = bc_loss + pde_loss
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

# Data preparation (simple sampling for training)
def generate_data(n_bc=2, n_colloc=100):
    # Domain between y = -h/2 and h/2
    h = 1.0
    y_bc = torch.tensor([[-h/2], [h/2]], dtype=torch.float32)  # BC points
    u_bc = torch.zeros_like(y_bc)                              # u=0 at plates
    y_colloc = torch.linspace(-h/2, h/2, n_colloc).reshape(-1,1).float()  # collocation points (PDE residual)

    return {'y_bc': y_bc, 'u_bc': u_bc, 'y_colloc': y_colloc}

# Lightning DataModule for clean batch management
class PINNDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.data = generate_data()

    def train_dataloader(self):
        # Return a data loader with batch size 1 that yields the full dict
        return torch.utils.data.DataLoader([self.data], batch_size=1)

# Instantiate model and data module
mu = 1.0    # dynamic viscosity
dpdx = -1.0 # constant pressure gradient

model = PINN(layers=[1, 20, 20, 1], mu=mu, dpdx=dpdx)
data_module = PINNDataModule()

# Trainer
trainer = pl.Trainer(max_epochs=600, accelerator='auto', log_every_n_steps=10)

# Train
trainer.fit(model, datamodule=data_module)


ClearML Task: created new task id=9f928c0b72e24ed0ac3c412c6e4e3508


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


2025-09-11 16:44:49,020 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/3b8e0332a8dc4f6ba4a5abbb428a0880/experiments/9f928c0b72e24ed0ac3c412c6e4e3508/output/log


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring


INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type       | Params | Mode 
--------------------------------------------------
0 | layers     | ModuleList | 481    | train
1 | activation | Tanh       | 0      | train
--------------------------------------------------
481       Trainable params
0         Non-trainable params
481       Total params
0.002     Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/loops/fit_loop.py:310: PossibleUserWarning:

The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=100). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.



Training: |          | 0/? [00:00<?, ?it/s]

ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=600` reached.


In [4]:
# Plot PINN vs Analytical
import numpy as np
h = data_module.data['h']
y_vals = np.linspace(-h/2, h/2, 200).reshape(-1,1).astype(np.float32)
y_torch = torch.tensor(y_vals)

model.eval()
with torch.no_grad():
    u_pred = model(y_torch).cpu().numpy().flatten()

u_analytical = analytical_solution(y_vals, mu, dpdx, h).flatten()

plt.figure(figsize=(8,5))
plt.plot(y_vals, u_pred, label='PINN Prediction', linewidth=2)
plt.plot(y_vals, u_analytical, '--', label='Analytical Solution', linewidth=2)
plt.xlabel('y')
plt.ylabel('Velocity u(y)')
plt.title('Velocity Profile between Parallel Plates')
plt.legend()
plt.grid(True)
plt.show()

KeyError: 'h'